<a href="https://colab.research.google.com/github/navami-b/LangChain-s-TextLoader/blob/main/textloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab cell
!pip install -q langchain langchain-community langchain-text-splitters pypdf transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [2]:
# Colab cell
from google.colab import files
uploaded = files.upload()   # choose your .txt or .pdf
# get the first uploaded filename
filepath = next(iter(uploaded.keys()))
print("Uploaded:", filepath)


Saving blind school.pdf to blind school.pdf
Uploaded: blind school.pdf


In [4]:
# 3) Load and split the document into chunks

# Try community loaders first (newer versions), fall back to core loaders if needed
try:
    from langchain_community.document_loaders import TextLoader, PyPDFLoader
except ImportError:
    from langchain.document_loaders import TextLoader, PyPDFLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_and_chunk(path: str, chunk_size: int = 1000, chunk_overlap: int = 200):
    """
    Loads a .txt or .pdf and returns a list of chunk strings.
    Prints counts.
    """
    ext = path.lower().split('.')[-1]

    # Select loader based on file type
    if ext == "txt":
        loader = TextLoader(path, encoding="utf-8", autodetect_encoding=True)
        docs = loader.load()
    elif ext == "pdf":
        loader = PyPDFLoader(path)
        docs = loader.load()
    else:
        raise ValueError("Unsupported file type. Use .txt or .pdf")

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )

    chunks = []
    for doc in docs:
        text = getattr(doc, "page_content", str(doc))

        if hasattr(splitter, "split_text"):
            parts = splitter.split_text(text)
            chunks.extend(parts)
        elif hasattr(splitter, "split_documents"):
            parts = splitter.split_documents([doc])
            for p in parts:
                chunks.append(getattr(p, "page_content", str(p)))
        elif hasattr(splitter, "create_documents"):
            parts = splitter.create_documents([text])
            for p in parts:
                chunks.append(getattr(p, "page_content", str(p)))
        else:
            raise RuntimeError("No compatible split method found on RecursiveCharacterTextSplitter")

    print(f"Loaded {len(docs)} document(s). Produced {len(chunks)} chunks (chunk_size={chunk_size}, overlap={chunk_overlap}).")
    return chunks

# Call the function
chunks = load_and_chunk(filepath, chunk_size=1000, chunk_overlap=200)

# Print total chunk count
print("Total chunks:", len(chunks))

# Show sample chunks (with newline replacement handled outside f-string)
print("\n--- sample chunks (first 2) ---")
for i, c in enumerate(chunks[:2], 1):
    preview = c[:300].replace("\n", " ")
    print(f"[chunk {i}] {preview}\n")


Loaded 2 document(s). Produced 4 chunks (chunk_size=1000, overlap=200).
Total chunks: 4

--- sample chunks (first 2) ---
[chunk 1] ReportonVisuallyImpairedSchoolsforMajorProject O B J E C T I V E  T o  g a t h e r  s o m e  i n s i g h t s  o n  t h e  c h a l l e n g e s  b l i n d  p e o p l e  e n c o u n t e r  a n d  t h e  t o o l s  c u r r e n t l y  i n  u s e  f o r  n a v i g a t i o n  t o  h e l p  u s  i m p r o v

[chunk 2] i m p r o v e m e n t  t o  a s c e r t a i n  r e a l - t i m e  d e t e c t i o n  o f  o b s t a c l e s . ● I t  i s  t o u g h  t o  n a v i g a t e  i n  c o n g e s t e d  p l a c e s  o r  w h e n  t h e y  a r e  u n f a m i l i a r  w e a t h e r  c o n d i t i o n s  l i k e  r a i n  p o

